In [ ]:
# Cài đặt thư viện cần thiết cho web scraping (chạy một lần)
# - beautifulsoup4: phân tích HTML
# - requests: gửi HTTP request
!pip3 install beautifulsoup4
!pip3 install requests

In [12]:
# ======================================
# 🚀 FPT UNIVERSITY - SpaceX Web Scraping Lab
# ======================================

# --- Step 0: Cài đặt thư viện (nếu cần) ---
# !pip install beautifulsoup4
# !pip install requests

# --- Step 1: Import các thư viện cần thiết ---
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata


# --- Step 2: Định nghĩa các hàm hỗ trợ (helper functions) ---

def date_time(table_cells):
    """
    Extract date and time from HTML table cell.
    """
    return [data_time.strip() for data_time in list(table_cells.stripped_strings)][0:2]


def booster_version(table_cells):
    """
    Extract booster version from HTML table cell.
    """
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.stripped_strings) if i % 2 == 0][0:-1])
    return out


def landing_status(table_cells):
    """
    Extract landing status from HTML table cell.
    """
    out = [i for i in table_cells.stripped_strings][0] if table_cells else None
    return out


def get_mass(table_cells):
    """
    Extract payload mass (kg) from HTML table cell.
    """
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if "kg" in mass:
        new_mass = mass[0:mass.find("kg") + 2]
    else:
        new_mass = None
    return new_mass


def extract_column_from_header(row):
    """
    Extract clean column name from HTML header row (<th>).
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    column_name = ' '.join(row.contents)
    if not (column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name


# --- Step 3: Đọc dữ liệu HTML từ Wikipedia ---
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(static_url, headers=headers)
print("Status Code:", response.status_code)

soup = BeautifulSoup(response.text, "html.parser")
print("Page Title:", soup.title.string if soup.title else "Không tìm thấy tiêu đề trang")


# --- TASK 2: Extract column headers ---
html_tables = soup.find_all('table', class_='wikitable')
print("Number of tables found:", len(html_tables))

first_launch_table = html_tables[0]
column_names = []

for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name:
        column_names.append(name)

print("Column names found:")
print(column_names)


# --- TASK 3: Tạo dictionary rỗng để lưu dữ liệu ---
launch_dict = dict.fromkeys(column_names)
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

for key in ['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 
            'Customer', 'Launch outcome', 'Version Booster', 
            'Booster landing', 'Date', 'Time']:
    launch_dict[key] = []


# --- TASK 4: Duyệt qua từng bảng và lấy dữ liệu ---
extracted_row = 0

for table_number, table in enumerate(soup.find_all('table', 'wikitable plainrowheaders collapsible')):
    for rows in table.find_all("tr"):

        # Kiểm tra xem hàng có số thứ tự không
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        row = rows.find_all('td')

        if flag and len(row) >= 8:
            extracted_row += 1

            # --- Flight number ---
            launch_dict['Flight No.'].append(flight_number)

            # --- Date & Time ---
            datetimelist = date_time(row[0])
            date = datetimelist[0].strip(',') if len(datetimelist) > 0 else None
            time = datetimelist[1] if len(datetimelist) > 1 else None
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # --- Booster version ---
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # --- Launch Site ---
            launch_site = row[2].a.string if len(row) > 2 and row[2].a else row[2].text.strip() if len(row) > 2 else None
            launch_dict['Launch site'].append(launch_site)

            # --- Payload ---
            payload = row[3].a.string if len(row) > 3 and row[3].a else row[3].text.strip() if len(row) > 3 else None
            launch_dict['Payload'].append(payload)

            # --- Payload Mass ---
            payload_mass = get_mass(row[4]) if len(row) > 4 else None
            launch_dict['Payload mass'].append(payload_mass)

            # --- Orbit ---
            orbit = row[5].a.string if len(row) > 5 and row[5].a else row[5].text.strip() if len(row) > 5 else None
            launch_dict['Orbit'].append(orbit)

            # --- Customer ---
            customer = row[6].a.string if len(row) > 6 and row[6].a else row[6].text.strip() if len(row) > 6 else None
            launch_dict['Customer'].append(customer)

            # --- Launch Outcome ---
            launch_outcome = list(row[7].strings)[0].strip() if len(row) > 7 and len(list(row[7].strings)) > 0 else None
            launch_dict['Launch outcome'].append(launch_outcome)

            # --- Booster Landing Outcome ---
            booster_landing = landing_status(row[8]) if len(row) > 8 else None
            launch_dict['Booster landing'].append(booster_landing)


print(f"\n✅ Total extracted rows: {extracted_row}")


# --- TASK 5: Tạo DataFrame và Xuất CSV ---
df = pd.DataFrame(launch_dict)
print("\nPreview of DataFrame:")
print(df.head())

df.to_csv('spacex_web_scraped.csv', index=False)
print("\n📁 File 'spacex_web_scraped.csv' has been created successfully!")


Status Code: 200
Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia
Number of tables found: 13
Column names found:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']

✅ Total extracted rows: 121

Preview of DataFrame:
  Flight No. Launch site                               Payload Payload mass  \
0          1       CCAFS  Dragon Spacecraft Qualification Unit         None   
1          2       CCAFS                                Dragon         None   
2          3       CCAFS                                Dragon       525 kg   
3          4       CCAFS                          SpaceX CRS-1     4,700 kg   
4          5       CCAFS                          SpaceX CRS-2     4,877 kg   

  Orbit Customer Launch outcome  Version Booster Booster landing  \
0   LEO   SpaceX        Success  F9 v1.07B0003.1         Failure   
1   LEO     NASA        Success  F9 v1.07B0004.1         Failure   
2   LEO     NASA     